## Walkthrough of model deployment as ML web service on Kubernetes

This notebook outlines steps for deploying a simple custom-built REST API prediction service to Kubernetes


### Prepare environment

**Import libraries**

In [1]:
from yaml import load, Loader
import os
import requests
import json

**Load config for environment variables**

Load configuration

In [29]:
with open('config.yaml','r') as config_file:
    config = load(config_file, Loader=Loader)

docker_registry = config['DOCKER_REGISTRY']
service_name = config['SERVICE_NAME']
api_version = config['API_VERSION']
model_repo = '..\experimentation\models'

Copy latest model to deployment directory

In [11]:
latest_model = sorted(os.listdir(model_repo))[-1]
latest_model_path = os.path.join(model_repo,latest_model)

!copy "{latest_model_path}" .

        1 file(s) copied.


### Containerise the prediction service using Docker

**Build the docker image**

Create a relevant tag that includes the image repository, a name for the service and its version.

In [14]:
tag = f'{docker_registry}/{service_name}:{api_version}'
!docker build -t {tag} .

#1 [internal] load build definition from Dockerfile
#1 sha256:e7adba2b8b13e7fdacb7553ff9bb73f0fb7240ab29a30b9f1568ef9e97968e3f
#1 transferring dockerfile: 225B 0.0s done
#1 DONE 0.0s

#2 [internal] load .dockerignore
#2 sha256:5938fa3fa74aa1e61ba1f82222caadf1863de426dfa850506ff8f869456c67a7
#2 transferring context: 2B done
#2 DONE 0.0s

#3 [internal] load metadata for docker.io/library/python:3.9-slim
#3 sha256:3425157df499c84dd49181e5611a11caeed16adf15a5ddbcfa4c3002c56d3d27
#3 DONE 3.9s

#4 [1/5] FROM docker.io/library/python:3.9-slim@sha256:f4efbe5d1eb52c221fded79ddf18e4baa0606e7766afe2f07b0b330a9e79564a
#4 sha256:9ce0d84a404c9ac604ef98baa1f1065d5a70e321684b314c01df3d72c5a89693
#4 resolve docker.io/library/python:3.9-slim@sha256:f4efbe5d1eb52c221fded79ddf18e4baa0606e7766afe2f07b0b330a9e79564a 0.0s done
#4 ...

#6 [internal] load build context
#6 sha256:37986f1db2c29fa8823ecaec8196ec4d3038b3ad1a4fca3ce4dec35c29fa2710
#6 transferring context: 4.54kB 0.0s done
#6 DONE 0.0s

#4 [1/5] FRO

**Run the service on Docker**

Run the image as a container locally and map container port 5000 to localhostport 5000 for testing.

In [ ]:
!docker run -it -p 5000:5000 --name test-ml-model edlongbottom/mlwebservice/titanic:0.0.1

**Test the service**

Use Curl or the python requests module to test the prediction web service

In [30]:
# example of an input that should yield 0
X1 = [-0.11338264511659583,-1.0524307127141423, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]

# example of an input that should yield 1
X2 = [-1.632430989759686,0.231855615029561,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0]

# define base URL (localhost + flask port)
url = f"http://127.0.0.1:5000/{service_name}/v{api_version}/predict"

In [26]:
# using Curl
!curl -X POST -H "Content-Type:application/json" --data "{\"X\":[-1.632430989759686,0.231855615029561,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0]}" http://127.0.0.1:5000/titanic/v0.0.1/predict

{"prediction":1}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   145  100    17  100   128     17    128  0:00:01 --:--:--  0:00:01  141k


In [31]:
# define requests body and headers
body={'X':X2}
headers = {"Content-Type": "application/json"}

# send a get request to flask api
response = requests.post(url=url, data=json.dumps(body), headers=headers)
print(response.json()) 

{'prediction': 1}


**Tear down**

Once testing is complete, stop and remove the docker container

In [ ]:
!docker stop test-ml-model
!docker rm test-ml-model

### Deploy the prediction service to Kubernetes

Push the image to Docker hub

In [ ]:
!docker push {tag}

Deploy the image to Kubernetes using Helm

In [16]:
!helm upgrade --install mlwebservice-titanic helm-ml-serving

Release "mlwebservice-titanic" does not exist. Installing it now.
NAME: mlwebservice-titanic
LAST DEPLOYED: Thu Dec 23 11:47:24 2021
NAMESPACE: default
STATUS: deployed
REVISION: 1
TEST SUITE: None


Test the web service using Python requests module

In [24]:
# example of an input that should yield 0
X1 = [-0.11338264511659583,-1.0524307127141423, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]

# example of an input that should yield 1
X2 = [-1.632430989759686,0.231855615029561,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0]

# define base URL (localhost + flask port)
url = f"http://127.0.0.1:5000/{service_name}/v{api_version}/predict"
body={'X':X2}
headers = {"Content-Type": "application/json"}

In [25]:
# send a get request to flask api
response = requests.post(url=url, data=json.dumps(body), headers=headers)
print(response.json()) 

{'prediction': 1}


Test the web service using Curl

In [23]:
url = f"http://127.0.0.1:5000/{service_name}/v{api_version}/predict"
!curl -X POST -H "Content-Type:application/json; format=pandas-split" --data "{\"columns\":[\"PassengerId\",\"Survived\",\"Pclass\",\"Name\",\"Sex\",\"Age\",\"SibSp\",\"Parch\",\"Ticket\",\"Fare\",\"Cabin\",\"Embarked\"],\"data\":[[892,3,'Kelly, Mr. James','male',34.5,0,0,'330911',7.8292,nan,'Q']]}" {url}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0curl: (6) Could not resolve host: url


**Tear down**

Remove the service when not in use

In [ ]:
!helm uninstall mlwebservice-titanic